In [ ]:
from distributed import Client
client = Client("dask_scheduler:8786")
client.get_versions(check=True)
client

In [ ]:
###### Please write your cuStreamz job below ####

In [ ]:
from streamz import Stream
import numpy as np
from streamz.dataframe import DataFrame
import time
import math
import confluent_kafka
import json
import os

In [ ]:
def gpu_preprocess_simple_agg(messages):
    import cudf
    preprocess_start_time = int(round(time.time()))
    size = len(messages)*len(messages[0])    
    json_input_string = "\n".join([msg.decode('utf-8') for msg in messages])
    pre_gpu_timestamp = int(round(time.time()))
    gdf = cudf.read_json(json_input_string, lines=True, engine='cudf')
    preprocess_end_time = int(round(time.time()))
    gdf['Count'] = 1
    num_rows = gdf['Count'].sum()
    gdf1 = gdf.groupby(['AppId0']).mean()
    agg_end_time = int(round(time.time()))
    return "{0},{1},{2},{3},{4},{5}".format(num_rows, preprocess_start_time, \
                                            pre_gpu_timestamp, preprocess_end_time, agg_end_time, size)

In [ ]:
topic = "custreamz-topic"

#Enter Kafka Pod's IP
bootstrap_servers = 'localhost:9092'

consumer_conf = {'bootstrap.servers': bootstrap_servers, \
                 'group.id': 'custreamz', 'session.timeout.ms': 60000}

stream = Stream.from_kafka_batched(topic, consumer_conf, poll_interval='5s', \
                                   npartitions=32, asynchronous = True, dask= True)

final_output = stream.map(gpu_preprocess_simple_agg).buffer(10000).gather().sink_to_list()

stream.start()

In [ ]:
final_output

In [ ]:
stream.destroy()